In [34]:
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
import hopsworks
import pandas as pd
import numpy as np

/Users/berkan/Desktop/wine_quality/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
#read the wine quality data set
wine_df = pd.read_csv("https://raw.githubusercontent.com/ID2223KTH/id2223kth.github.io/master/assignments/lab1/wine.csv")
wine_df

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [36]:
wine_df['type'].value_counts()

type
white    4898
red      1599
Name: count, dtype: int64

In [37]:
numerical_col = wine_df.select_dtypes(np.number).columns  #list(wine_df.columns[1:11])
categorical_col = wine_df.select_dtypes(object).columns #["type", "quality"]
wine_df[numerical_col] = wine_df[numerical_col].fillna(wine_df[numerical_col].mean())
wine_df["type"] = wine_df["type"].fillna("unknown")

In [38]:
#-Transform categorical variables into numerical variables
wine_df['type'].replace(['white', 'red', 'unknown'], [0, 1, 2], inplace=True)
wine_df.describe()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,0.246114,7.216579,0.339691,0.318722,5.444326,0.056042,30.525319,115.744574,0.994697,3.218395,0.531215,10.491801,5.818378
std,0.430779,1.295751,0.164548,0.145231,4.757392,0.035031,17.749400,56.521855,0.002999,0.160637,0.148768,1.192712,0.873255
min,0.000000,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,0.000000,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,0.000000,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,0.000000,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,1.000000,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [39]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
# Use mutual information for feature selection 
selector = SelectKBest(mutual_info_classif, k=10)
X = wine_df.drop('quality', axis=1)
y = wine_df['quality']
X_selected = selector.fit_transform(X, y)

# Get selected feature indices
selected_indices = selector.get_support()

# Get the selected feature names
selected_features = X.columns[selected_indices]
dropped_features = X.columns[~selected_indices]

# Print the selected features
print("Selected features:", selected_features)
print("Dropped features:", dropped_features)

Selected features: Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'sulphates', 'alcohol'],
      dtype='object')
Dropped features: Index(['type', 'pH'], dtype='object')


In [40]:
wine_df = wine_df.drop(dropped_features, axis=1)
wine_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,0.450000,8.8,6
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,0.490000,9.5,6
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,0.440000,10.1,6
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,0.400000,9.9,6
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,0.400000,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,0.580000,10.5,5
6493,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,0.531215,11.2,6
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,0.750000,11.0,6
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,0.710000,10.2,5


In [41]:
wine_df.columns = wine_df.columns.str.replace(' ', '_')
wine_df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,sulphates,alcohol,quality
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,0.450000,8.8,6
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,0.490000,9.5,6
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,0.440000,10.1,6
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,0.400000,9.9,6
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,0.400000,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,0.580000,10.5,5
6493,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,0.531215,11.2,6
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,0.750000,11.0,6
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,0.710000,10.2,5


Write the features to the feature store as a Feature Group

In [42]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login(api_key_value=api_key, project="DD2223_lab1")
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/206996
Connected. Call `.close()` to terminate connection gracefully.


In [112]:
random_quality = np.random.randint(3,10)

4

In [143]:
distinct_features  = set()
for i in result.index:
    distinct_features.add(i[0])


In [147]:
made_up_data = {}
for feature in distinct_features:
    mean,std = result.iloc[:,0][feature]
    value = np.random.normal(mean,std,1)
    if value < 0:
        value = np.abs(value)
    made_up_data[feature] = value

made_up_data

{'free_sulfur_dioxide': array([97.3812858]),
 'residual_sugar': array([7.53206759]),
 'total_sulfur_dioxide': array([263.11696415]),
 'alcohol': array([10.04846945]),
 'fixed_acidity': array([9.31437989]),
 'density': array([0.99784368]),
 'citric_acid': array([0.22246415]),
 'volatile_acidity': array([0.64644329]),
 'sulphates': array([0.27164506]),
 'chlorides': array([0.12933479])}

In [44]:
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

def expect(suite, column, min_val, max_val):
    suite.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column":column, 
            "min_value":min_val,
            "max_value":max_val,
        }
    )
)

In [46]:
suite = ExpectationSuite(expectation_suite_name="wine_quality_dimensions")
expect(suite, 'fixed_acidity', 3.8, 15.9)
expect(suite, 'volatile_acidity', 0.08, 1.58)             
expect(suite, 'citric_acid', 0.0, 1.66)
expect(suite, 'residual_sugar', 0.6, 65.8)             
expect(suite, 'chlorides', 0.009, 0.611)
expect(suite, 'free_sulfur_dioxide', 1.0, 289.0)
expect(suite, 'total_sulfur_dioxide', 6.0, 440.0)
expect(suite, 'density', 0.98711, 1.03898)
expect(suite, 'sulphates', 0.22, 2.0)
expect(suite, 'alcohol', 8.0, 14.9)
wine_fg.save_expectation_suite(expectation_suite=suite, validation_ingestion_policy="STRICT")  

Attached expectation suite to Feature Group, edit it at https://c.app.hopsworks.ai:443/p/206996/fs/206915/fg/237947
